### Training a neural network to memorize the patterns using Hopfield Network

Import the required packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

Declare the patterns that our network can memorize

||||||
|-------||-------||-------------|
|1    || 1  ||1          |
|0    ||1   ||0         |
|0    ||1   ||0        |

||||||
|-------||-------||-------------|
|1    || 0 ||1          |
|1    ||0  ||1         |
|1    ||1   ||1        |

Replace the zeroes with negative ones to calculate the weight matrix. These patterns are constructed as 9x1 matrix

Calculate the W using dot product of both the patterns

In [2]:
pattern_1=[1, 1, 1, -1, 1, -1, -1, 1, -1]
pattern_2=[1, -1, 1, 1, -1, 1, 1, 1, 1]
p1 = np.resize(np.array(pattern_1), (1,9))
p2 = np.resize(np.array(pattern_2), (1,9))
w=np.dot(np.resize(p1,(9,1)), p1) + np.dot(np.resize(p2,(9,1)), p2)
w=w-(2*np.identity(9, dtype=int))
w

array([[ 0,  0,  2,  0,  0,  0,  0,  2,  0],
       [ 0,  0,  0, -2,  2, -2, -2,  0, -2],
       [ 2,  0,  0,  0,  0,  0,  0,  2,  0],
       [ 0, -2,  0,  0, -2,  2,  2,  0,  2],
       [ 0,  2,  0, -2,  0, -2, -2,  0, -2],
       [ 0, -2,  0,  2, -2,  0,  2,  0,  2],
       [ 0, -2,  0,  2, -2,  2,  0,  0,  2],
       [ 2,  0,  2,  0,  0,  0,  0,  0,  0],
       [ 0, -2,  0,  2, -2,  2,  2,  0,  0]])

Declare some initial patterns to test the network

In [3]:
patterns = [[1, 1, 1, 0, 0, 1, 1, 0, 1], [1, 0, 0, 1, 0, 1, 0, 0, 1], [1, 1, 0, 0, 1, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 1, 0, 1, 1], [1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 0, 1, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 1, 1], [1, 1, 0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0],
 [0, 1, 0, 1, 1, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1], [0, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 1, 1, 1, 1, 1, 0], [1, 1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 0, 1, 0, 1, 0, 0, 0],
 [1, 0, 0, 1, 1, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 1, 0, 1], [1, 0, 0, 1, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 1, 0, 1, 1, 1], [1, 0, 0, 1, 1, 0, 0, 1, 0], [1, 1, 1, 0, 1, 0, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0, 1], [0, 1, 0, 0, 1, 0, 1, 1, 1],
 [1, 0, 0, 1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 0, 1, 1, 1, 1]]

Table is data structure to save the initial state, final state and number of iterations for each pattern.

In [4]:
class Table:
    def __init__(self, initial_state, final_state, iterations):
        self.initial_state = initial_state
        self.final_state = final_state
        self.iterations = iterations

Activation method that is applied to weight matrix.

In [5]:
def sign(pattern_new, pattern_old):
    for i in range(0, len(pattern_new)):
        if pattern_new[i] > 0 :
            pattern_new[i] = 1
        elif pattern_new[i] < 0:
            pattern_new[i] = 0
        else:
            pattern_new[i] = pattern_old[i]
    return pattern_new

Synchronous weight update method where the pattern is updated all at once.

In [6]:
def synchronous_update(pattern, weights, epochs=10, stable_threshold=3, verbose=True) :
    local_states=[]
    stable=1
    p=np.resize(pattern,(9,1))
    if verbose:
        print(pattern)
    for iteration in range(0,epochs):
        p_old = p
        p=np.dot(weights, p)
        p=sign(np.ravel(p), np.ravel(p_old))
        if verbose:
            print("Iteration-", iteration)
            print("==================")
            print(p)
        if (np.array_equal(p, p_old) and stable == stable_threshold) :
            break;
        if np.array_equal(p, p_old):
            stable = stable +1;
        local_states.append(p)
    table=Table(pattern, np.resize(p, (1,9)), iteration)
    sync_table_data.append(table)

Asynchronous weight update method where the each element pattern selected at random is updated.

In [7]:
def asynchronous_update(pattern, weights, random_seed=5, epochs=1000, 
                        stable_threshold=3, verbose=True) :
    np.random.seed(random_seed)
    local_states=[]
    stable=1
    p=np.resize(pattern,(9,1))
    if verbose:
        print(pattern)
    for iteration in range(0,epochs):
        p_old = p
        dot=np.dot(weights, p)
        i=np.random.randint(0,9)
        if dot[i] < 0:
            p[i] = 0
        elif dot[i] > 0:
            p[i] = 1
        else:
            p[i] = p_old[i]
        if verbose:
            print("Iteration-", iteration)
            print("==================")
            print(p)
        if (np.array_equal(p, p_old) and stable == stable_threshold) :
            break;
        if np.array_equal(p, p_old):
            stable = stable +1;
    local_states.append(p)
    table=Table(pattern, np.resize(p, (1,9)), iteration)
    async_table_data.append(table)

Helper method to print the initial state, final state and number of iterations in each pattern in tabular format

In [8]:
def print_table_data(data, three_by_three=False) :
    state=1;
    table_data=[]
    for table_row in data:
        if three_by_three : 
            table_data.append([state,
                               np.resize(table_row.initial_state, (3,3)), 
                               np.resize(table_row.final_state, (3,3)), 
                               table_row.iterations])
        else :
            table_data.append([state,
                               table_row.initial_state,
                               np.ravel(table_row.final_state),
                               table_row.iterations])
        state=state+1
    print(tabulate(table_data, headers=["State", "Initial State", 
                                        "Final State", "Iterations"]))

Run the synchronous update and asynchronus update methods to evolve to the patterns that network memorized.

In [9]:
sync_table_data=[]
async_table_data=[]
for p in patterns:
    synchronous_update(pattern=p, weights=w, 
                       stable_threshold=6, epochs=30,  verbose=False)
for p in patterns:
    asynchronous_update(pattern=p, weights=w, 
                        stable_threshold=30, epochs=100, verbose=False)

Print the initial state, final state and number of iterations to achieve it using synchronous update method

In [10]:
print_table_data(sync_table_data)

  State  Initial State                Final State            Iterations
-------  ---------------------------  -------------------  ------------
      1  [1, 1, 1, 0, 0, 1, 1, 0, 1]  [1 0 1 1 0 1 1 1 1]             6
      2  [1, 0, 0, 1, 0, 1, 0, 0, 1]  [1 0 1 1 0 1 1 1 1]             6
      3  [1, 1, 0, 0, 1, 0, 0, 0, 1]  [1 1 1 0 1 0 0 1 0]             6
      4  [0, 0, 0, 1, 0, 1, 0, 1, 1]  [1 0 1 1 0 1 1 1 1]             6
      5  [1, 0, 0, 1, 0, 0, 0, 0, 0]  [1 0 1 1 0 1 1 1 1]             6
      6  [1, 1, 0, 1, 1, 0, 1, 1, 1]  [1 0 1 1 0 1 1 1 1]             6
      7  [0, 0, 1, 0, 0, 0, 1, 1, 1]  [1 0 1 1 0 1 1 1 1]             6
      8  [1, 1, 0, 1, 0, 0, 0, 1, 0]  [1 0 1 0 0 0 0 1 0]             6
      9  [0, 1, 0, 1, 0, 1, 0, 0, 0]  [0 0 0 1 0 1 1 0 1]             6
     10  [0, 1, 0, 1, 1, 0, 1, 0, 0]  [0 0 0 0 0 0 0 0 0]             6
     11  [0, 0, 1, 0, 1, 0, 0, 0, 1]  [1 0 1 0 0 0 0 1 0]             6
     12  [0, 1, 0, 1, 1, 1, 0, 1, 1]  [1 0 1 1 0 1 1 1 1]       

Print the initial state, final state and number of iterations to achieve it using asynchronous update method

In [11]:
print_table_data(async_table_data)

  State  Initial State                Final State            Iterations
-------  ---------------------------  -------------------  ------------
      1  [1, 1, 1, 0, 0, 1, 1, 0, 1]  [1 0 1 1 0 1 1 1 1]            29
      2  [1, 0, 0, 1, 0, 1, 0, 0, 1]  [1 0 1 1 0 1 1 1 1]            29
      3  [1, 1, 0, 0, 1, 0, 0, 0, 1]  [1 1 1 0 1 0 0 1 0]            29
      4  [0, 0, 0, 1, 0, 1, 0, 1, 1]  [1 0 1 1 0 1 1 1 1]            29
      5  [1, 0, 0, 1, 0, 0, 0, 0, 0]  [1 0 1 1 0 1 1 1 1]            29
      6  [1, 1, 0, 1, 1, 0, 1, 1, 1]  [1 0 1 1 0 1 1 1 1]            29
      7  [0, 0, 1, 0, 0, 0, 1, 1, 1]  [1 0 1 1 0 1 1 1 1]            29
      8  [1, 1, 0, 1, 0, 0, 0, 1, 0]  [1 1 1 0 1 0 0 1 0]            29
      9  [0, 1, 0, 1, 0, 1, 0, 0, 0]  [0 0 0 1 0 1 1 0 1]            29
     10  [0, 1, 0, 1, 1, 0, 1, 0, 0]  [0 1 0 0 1 0 0 0 0]            29
     11  [0, 0, 1, 0, 1, 0, 0, 0, 1]  [1 1 1 0 1 0 0 1 0]            29
     12  [0, 1, 0, 1, 1, 1, 0, 1, 1]  [1 0 1 1 0 1 1 1 1]       

### All states in 3 x 3 matrix

Prepare all the possible patterns 

In [12]:
from itertools import product
m=3
n=3
x = product([1, 0], repeat=n*m)
x = np.reshape(list(x), (-1, n, m))

In [13]:
all_patterns=[]
for i in range(0,len(x)):
    all_patterns.append(np.ravel(x[i]))

Update using both sync and async update methods

In [14]:
sync_table_data=[]
async_table_data=[]
for p in all_patterns:
    synchronous_update(pattern=p, weights=w, 
                       stable_threshold=6, epochs=100,  verbose=False)
for p in all_patterns:
    asynchronous_update(pattern=p, weights=w, 
                        stable_threshold=15, epochs=100, verbose=False)

Print the results in the both the methods

If we look at the first pattern [1 1 1 1 1 1 1 1 1] it reached the final state [ 1 0 1 1 0 1  1  1  1] which is closer to stored patterns in network

In [15]:
print_table_data(sync_table_data)

  State  Initial State        Final State            Iterations
-------  -------------------  -------------------  ------------
      1  [1 1 1 1 1 1 1 1 1]  [1 0 1 1 0 1 1 1 1]             6
      2  [1 1 1 1 1 1 1 1 0]  [1 0 1 1 0 1 1 1 1]             6
      3  [1 1 1 1 1 1 1 0 1]  [1 0 1 1 0 1 1 1 1]             6
      4  [1 1 1 1 1 1 1 0 0]  [1 0 1 1 0 1 1 1 1]             6
      5  [1 1 1 1 1 1 0 1 1]  [1 0 1 1 0 1 1 1 1]             6
      6  [1 1 1 1 1 1 0 1 0]  [1 0 1 0 0 0 0 1 0]             6
      7  [1 1 1 1 1 1 0 0 1]  [1 0 1 1 0 1 1 1 1]             6
      8  [1 1 1 1 1 1 0 0 0]  [1 0 1 0 0 0 0 1 0]             6
      9  [1 1 1 1 1 0 1 1 1]  [1 0 1 1 0 1 1 1 1]             6
     10  [1 1 1 1 1 0 1 1 0]  [1 0 1 0 0 0 0 1 0]             6
     11  [1 1 1 1 1 0 1 0 1]  [1 0 1 1 0 1 1 1 1]             6
     12  [1 1 1 1 1 0 1 0 0]  [1 0 1 0 0 0 0 1 0]             6
     13  [1 1 1 1 1 0 0 1 1]  [1 0 1 0 0 0 0 1 0]             6
     14  [1 1 1 1 1 0 0 1 0]  [1 1 1 0 1

In [16]:
print_table_data(async_table_data)

  State  Initial State        Final State            Iterations
-------  -------------------  -------------------  ------------
      1  [1 1 1 1 1 1 1 1 1]  [1 0 1 1 0 1 1 1 1]            14
      2  [1 1 1 1 1 1 1 1 0]  [1 0 1 1 0 1 1 1 1]            14
      3  [1 1 1 1 1 1 1 0 1]  [1 0 1 1 0 1 1 1 1]            14
      4  [1 1 1 1 1 1 1 0 0]  [1 0 1 1 0 1 1 1 1]            14
      5  [1 1 1 1 1 1 0 1 1]  [1 0 1 1 0 1 1 1 1]            14
      6  [1 1 1 1 1 1 0 1 0]  [1 1 1 0 1 0 0 1 0]            14
      7  [1 1 1 1 1 1 0 0 1]  [1 0 1 1 0 1 1 1 1]            14
      8  [1 1 1 1 1 1 0 0 0]  [1 1 1 0 1 0 0 1 0]            14
      9  [1 1 1 1 1 0 1 1 1]  [1 0 1 1 0 1 1 1 1]            14
     10  [1 1 1 1 1 0 1 1 0]  [1 1 1 0 1 0 0 1 0]            14
     11  [1 1 1 1 1 0 1 0 1]  [1 0 1 1 0 1 1 1 1]            14
     12  [1 1 1 1 1 0 1 0 0]  [1 1 1 0 1 0 0 1 0]            14
     13  [1 1 1 1 1 0 0 1 1]  [1 1 1 0 1 0 0 1 0]            14
     14  [1 1 1 1 1 0 0 1 0]  [1 1 1 0 1